In [35]:
! pip install tdqm


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [36]:
! pip install xgboost


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
import scipy
from tqdm import tqdm
from xgboost import XGBClassifier
from utils import LoadRumoursDataset
import numpy as np

In [3]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch
mlflow.set_experiment("ML Models 2024-08-17")

2024/08/17 13:04:45 INFO mlflow.tracking.fluent: Experiment with name 'ML Models 2024-08-17' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/rumour-detection-pheme/mlruns/6', creation_time=1723899885838, experiment_id='6', last_update_time=1723899885838, lifecycle_stage='active', name='ML Models 2024-08-17', tags={}>

In [4]:
#
file_path_replies = r"/workspaces/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/workspaces/rumour-detection-pheme/posts_charlie_hebdo.pkl"


In [5]:
time_cut =1e6
processor = LoadRumoursDataset(file_path_replies, file_path_posts, time_cut)
processor.load_data()
processor.process_data()
df_final = processor.get_final_dataframe()

X  = df_final.drop(columns=['rumour'])
X = np.hstack([X.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X.embeddings_avg.tolist()))])
y =df_final['rumour']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42,stratify=y
)

In [6]:
df_final

,followers,favorite_count,retweet_count,rumour,embeddings_avg,replies,first_time_diff,no_verified,verified
0,-0.073004,-0.532290,-0.160000,1,"[-0.12270056130364537, 0.01583862374536693, -0...",5,3.510619,1,0
1,0.031065,-0.344423,1.293333,1,"[-0.12335950043052435, -0.055849663292368255, ...",5,0.999077,0,1
2,0.356672,-0.524462,-0.302222,1,"[-0.1364929385483265, -0.07159566258390744, -0...",5,0.740536,0,1
3,0.332156,-0.524462,-0.400000,1,"[-0.045377860377941816, -0.20127306692302227, ...",3,-0.164358,0,1
4,1.046022,-0.391389,0.964444,1,"[-0.03706469060853124, -0.1309182441327721, -0...",10,-0.352724,0,1
...,...,...,...,...,...,...,...,...,...
1997,3.895910,-0.140900,-0.124444,0,"[0.21622000262141228, -0.15450449846684933, -0...",8,0.204986,0,1
1998,10.452911,3.859100,5.213333,0,"[0.21485890651291067, 0.03315381561829285, -0....",9,-0.245614,0,1
1999,0.012322,1.189824,0.955556,0,"[0.08846200071275234, -0.1485882457345724, 0.1...",9,0.500462,1,0
2000,3.631498,0.477495,1.075556,0,"[0.021962551607025996, -0.019428667094972398, ...",18,-0.271468,0,1


In [17]:
def plot_and_log_curves(history, metric_name, title):
    plt.figure(figsize=(10, 6))
    plt.plot(history['train'], label='Training')
    plt.plot(history['val'], label='Validation')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel(metric_name)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{metric_name}_curve.png')
    plt.close()
    mlflow.log_artifact(f'{metric_name}_curve.png')

In [13]:
import mlflow
import mlflow.sklearn
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

def train_optimize_model_with_mlflow(model, param_grid, cv, scoring, X_train, y_train, X_test, y_test,time_cut):
    """
    Train a Gradient Boosting Classifier with GridSearchCV and log the results with MLflow.

    Parameters:
    - model: The machine learning model (e.g., GradientBoostingClassifier()).
    - param_grid: The grid of parameters to search over.
    - cv: The cross-validation strategy (e.g., 5 for 5-fold cross-validation).
    - scoring: The scoring method for model evaluation (e.g., 'f1').
    - X_train: Training data features.
    - y_train: Training data labels.
    - X_test: Test data features.
    - y_test: Test data labels.
    """
    
    # Setup the grid search
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring=scoring, verbose=2, n_jobs=2)

    # Start an MLflow run
    with mlflow.start_run():

        # Set the tag with the model name
        mlflow.set_tag("model_name", model.__class__.__name__)
        
        # Perform the grid search
        grid_search.fit(X_train, y_train)

        # Get the best parameters
        best_params = grid_search.best_params_
        print(f"Best parameters: {best_params}")

        # Train the best model
        best_model = grid_search.best_estimator_
        best_model.fit(X_train, y_train)

        # Predict and evaluate
        y_train_pred = best_model.predict(X_train)
        y_test_pred = best_model.predict(X_test)

        # Calculate metrics
        metrics = {
            "train_recall": recall_score(y_train, y_train_pred, average='macro'),
            "train_precision": precision_score(y_train, y_train_pred, average='macro'),
            "train_f1_score": f1_score(y_train, y_train_pred, average='macro'),
            "train_accuracy": accuracy_score(y_train, y_train_pred),
            "test_recall": recall_score(y_test, y_test_pred, average='macro'),
            "test_precision": precision_score(y_test, y_test_pred, average='macro'),
            "test_f1_score": f1_score(y_test, y_test_pred, average='macro'),
            "test_accuracy": accuracy_score(y_test, y_test_pred)
        }

        # Log parameters and metrics
        mlflow.log_params(best_params)
        mlflow.log_param("time_cut",time_cut)
        mlflow.log_metrics(metrics)

        # Log the model
        mlflow.sklearn.log_model(best_model, "model")
        
        print("best params: \n",best_params)
        print("Metrics: \n",metrics)

        return best_params

    # End the MLflow run (this happens automatically when the 'with' block ends)


#### Gradient boosting

In [14]:
# Define the model
gbc = GradientBoostingClassifier()

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.001, 0.01,0.1],
    'max_depth': [3, 10,15],
    #'subsample': [0.7, 0.8],
    'min_samples_split': [2, 5]
}

cv = 5 
scoring = 'f1'

best_params =train_optimize_model_with_mlflow(gbc, param_grid, cv, scoring, X_train, y_train, X_test, y_test,time_cut)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best parameters: {'learning_rate': 0.1, 'n_estimators': 200}


/usr/local/python/3.10.13/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/usr/local/python/3.10.13/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/08/17 13:13:17 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


best params: 
 {'learning_rate': 0.1, 'n_estimators': 200}
Metrics: 
 {'train_recall': 0.9957983193277311, 'train_precision': 0.9987971130713713, 'train_f1_score': 0.9972881275545551, 'train_accuracy': 0.9981261711430356, 'test_recall': 0.8042515183994283, 'test_precision': 0.8117358518193133, 'test_f1_score': 0.8078824858757062, 'test_accuracy': 0.8678304239401496}


In [16]:
import mlflow
import numpy as np
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

def train_and_log_model(model, best_params, X_train, y_train, X_test, y_test, time_cut):
    # Define the model
    
    # Set the best parameters
    model.set_params(**best_params)
    
    # Start an MLflow run
    with mlflow.start_run():
        # Set the tag with the model name
        mlflow.set_tag("model_name", model.__class__.__name__)

        # Train the model
        model.fit(X_train, y_train)

        # Predict and evaluate
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Calculate metrics
        metrics = {
            "train_recall": recall_score(y_train, y_train_pred, average='macro'),
            "train_precision": precision_score(y_train, y_train_pred, average='macro'),
            "train_f1_score": f1_score(y_train, y_train_pred, average='macro'),
            "train_accuracy": accuracy_score(y_train, y_train_pred),
            "test_recall": recall_score(y_test, y_test_pred, average='macro'),
            "test_precision": precision_score(y_test, y_test_pred, average='macro'),
            "test_f1_score": f1_score(y_test, y_test_pred, average='macro'),
            "test_accuracy": accuracy_score(y_test, y_test_pred)
        }

        # Log parameters and metrics
        mlflow.log_params(best_params)
        mlflow.log_metrics(metrics)
        mlflow.log_param("time_cut", time_cut)

        # Log the model with example input
        example_input = np.array(X_train[0]).reshape(1, -1)  # Use the shape of your feature set

        # Log the model
        mlflow.sklearn.log_model(model, "model", input_example=example_input)

        print("Metrics: \n", metrics)


[CV] END ..............learning_rate=0.001, n_estimators=100; total time=   4.8s
[CV] END ..............learning_rate=0.001, n_estimators=100; total time=   4.4s
[CV] END ..............learning_rate=0.001, n_estimators=200; total time=   9.2s
[CV] END ..............learning_rate=0.001, n_estimators=200; total time=   9.1s
[CV] END ..............learning_rate=0.001, n_estimators=200; total time=   9.3s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=   4.6s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=   4.6s
[CV] END ...............learning_rate=0.01, n_estimators=200; total time=   9.1s
[CV] END ...............learning_rate=0.01, n_estimators=200; total time=   9.2s
[CV] END ................learning_rate=0.1, n_estimators=100; total time=   4.8s
[CV] END ................learning_rate=0.1, n_estimators=100; total time=   4.7s
[CV] END ................learning_rate=0.1, n_estimators=100; total time=   4.7s
[CV] END ................lea

In [73]:

# Example usage:

# Define the model
model = GradientBoostingClassifier()
time_cut =1e6

train_and_log_model(model,best_params, X_train, y_train, X_test, y_test,time_cut)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/python/3.10.13/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that set

Metrics: 
 {'train_recall': 0.5, 'train_precision': 0.3885071830106184, 'train_f1_score': 0.4372583479789104, 'train_accuracy': 0.7770143660212367, 'test_recall': 0.5, 'test_precision': 0.3877805486284289, 'test_f1_score': 0.43679775280898875, 'test_accuracy': 0.7755610972568578}


In [ ]:
# Initialize and train Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(**best_params)
gb_classifier.fit(X_train, y_train)
gb_pred = gb_classifier.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
gb_precision = precision_score(y_test, gb_pred, average='macro')
gb_recall = recall_score(y_test, gb_pred, average='macro')

In [ ]:
print("GB Accuracy:", gb_accuracy)
print("GB Precision:", gb_precision)
print("GB Recall:", gb_recall)

In [33]:

xgb = XGBClassifier(eval_metric='logloss')

# Train the best model
xgb.fit(X_train, y_train)

# Predict and evaluate
xgb_pred = xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred, average='macro')
xgb_recall = recall_score(y_test, xgb_pred, average='macro')

In [35]:
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)

XGB Accuracy: 0.8653366583541147
XGB Precision: 0.8111667558681857
XGB Recall: 0.790800285816363


In [ ]:
# Define the model
xgb = XGBClassifier(eval_metric='logloss')

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9, 1.0]
    #'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    #'min_child_weight': [1, 3, 5]
}

# Create a StratifiedKFold object with tqdm progress bar
cv = StratifiedKFold(n_splits=5)
cv_splits = list(cv.split(X_train, y_train))
tqdm_cv_splits = tqdm(cv_splits, total=len(cv_splits), desc="Grid Search Progress")

# Setup the grid search with the tqdm progress bar
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=tqdm_cv_splits, scoring='f1', verbose=2, n_jobs=-1)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Train the best model
best_xgb = grid_search.best_estimator_
best_xgb.fit(X_train, y_train)

# Predict and evaluate
y_pred = best_xgb.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=1))

Grid Search Progress: 100%|█████████████████████████████| 5/5 [00:00<00:00, 2305.07it/s]

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.7; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.7; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=

In [ ]:
best_params

In [18]:
if mlflow.active_run():
    mlflow.end_run()
